In [1]:
import os
os.remove('snorkel.db')

In [2]:
from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path=os.environ['SNORKELHOME'] + '/test/data/table_test/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [4]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%

CPU times: user 272 ms, sys: 29.1 ms, total: 301 ms
Wall time: 955 ms
34 Phrases in Corpus (Sandbox)


In [5]:
# for phrase in corpus.documents[0].phrases: print phrase

In [6]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
# Disease = candidate_subclass('Disease', ['disease'])
Disease_Year = candidate_subclass('Disease_Year', ['disease','year'])

In [7]:
from snorkel.candidates import OmniNgrams

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)

In [8]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 
              'brain cancer', 'disease', 'location', 'polio', 
              'chicken pox', 'plague', 'scurvy', 'infectious diseases', 
              'arthritis', 'yellow fever', 'fever', 'hypochondria', 'pneumonia']
print "Loaded %d diseases." % len(diseases)

Loaded 18 diseases.


In [9]:
from snorkel.matchers import DictionaryMatch, RegexMatchEach, DateMatcher

disease_matcher = DictionaryMatch(d=diseases, ignore_case=True)
year_matcher = DateMatcher()

In [10]:
from snorkel.candidates import CandidateExtractor

# ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
ce = CandidateExtractor(Disease_Year, 
                        [omni_ngrams, omni_ngrams], 
                        [disease_matcher, year_matcher])

In [11]:
%time candidates = ce.extract(corpus.documents, 'Sandbox Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 102 ms, sys: 7.02 ms, total: 109 ms
Wall time: 108 ms
Candidate Set (Sandbox Candidates) contains 48 Candidates


In [12]:
for c in candidates[:5]:
    print c

Disease_Year(Span("Location", parent=35, chars=[0,7], words=[0,0]), Span("1901", parent=48, chars=[0,3], words=[0,0]))
Disease_Year(Span("Location", parent=35, chars=[0,7], words=[0,0]), Span("2001", parent=45, chars=[0,3], words=[0,0]))
Disease_Year(Span("Location", parent=35, chars=[0,7], words=[0,0]), Span("1914", parent=39, chars=[0,3], words=[0,0]))
Disease_Year(Span("Hypochondria", parent=59, chars=[0,11], words=[0,0]), Span("1901", parent=48, chars=[0,3], words=[0,0]))
Disease_Year(Span("Hypochondria", parent=59, chars=[0,11], words=[0,0]), Span("2001", parent=45, chars=[0,3], words=[0,0]))


In [13]:
# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

In [14]:
# %time F_train = feature_manager.create(session, candidates, 'Sandbox Features')

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 14.5 s, sys: 135 ms, total: 14.6 s
Wall time: 14.7 s


In [18]:
# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(4))[:10]: print f

In [16]:
from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
from snorkel.lf_helpers import contains_token, contains_regex
from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
from snorkel.lf_helpers import _get_aligned_cells_better

In [19]:
c = candidates[8]
print c
print c.disease
print c.year
# print get_row_ngrams(c.year)
# print get_row_ngrams(c.year, infer=True)
_get_aligned_cells_better(c.year, 'col')

Disease_Year(Span("Cancer", parent=3, chars=[6,11], words=[1,1]), Span("1914", parent=39, chars=[0,3], words=[0,0]))
Span("Cancer", parent=3, chars=[6,11], words=[1,1])
Span("1914", parent=39, chars=[0,3], words=[0,0])


NameError: global name 'SnorkelSession' is not defined